In [ ]:
# !pip install tensorflow-datasets
# pip install --upgrade Pillow
# !pip install split_folders

In [ ]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
dir_food = '/home/davide/Documents/Progetto/dataset/FoodSplitted'

train_data_dir = dir_food + '/train'
valid_data_dir = dir_food + '/val'
test_data_dir = dir_food + '/test'

In [ ]:
img_height, img_width = (256, 256)
batch_size = 32

In [ ]:
train_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical') # set as training data

valid_generator = train_datagen.flow_from_directory(
    valid_data_dir, 
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical') # set as validation data

test_generator = train_datagen.flow_from_directory(
    test_data_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical') # set as test data

In [ ]:
# train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
#                                    shear_range = 0.2,
#                                    zoom_range = 0.2,
#                                    horizontal_flip = True,
#                                    validation_split = 0.4)

# train_generator = train_datagen.flow_from_directory(
#     train_data_dir,
#     target_size = (img_height, img_width),
#     batch_size = batch_size,
#     class_mode = 'categorical',
#     subset = 'training') # set as training data

# valid_generator = train_datagen.flow_from_directory(
#     valid_data_dir, 
#     target_size = (img_height, img_width),
#     batch_size = batch_size,
#     class_mode = 'categorical',
#     subset = 'validation') # set as validation data

# test_generator = train_datagen.flow_from_directory(
#     train_data_dir,
#     target_size = (img_height, img_width),
#     batch_size = 1,
#     class_mode = 'categorical',
#     subset = 'validation') # set as test data

In [ ]:
x, y = test_generator.next()
x.shape

In [ ]:
base_model = ResNet50(include_top=False, weights='imagenet')

In [ ]:
base_model.summary()

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
# x = Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.25)(x)
# x = Dense(256, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
  layer.trainable = False


In [ ]:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

In [ ]:
# base_learning_rate = 0.0001
# model.compile(optimizer = tf.keras.optimizers.Adam(lr = base_learning_rate),
#                                                 loss = 'categorical_crossentropy',
#                                                 metrics = ['accuracy'])

RMSprop = tf.keras.optimizers.RMSprop(
    learning_rate=0.001, rho=0.9, momentum=0.0, epsilon=1e-07, centered=False,
    name='RMSprop'
)

model.compile(optimizer = RMSprop, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(train_generator, epochs = 5, validation_data = valid_generator)

In [ ]:
test_loss, test_acc = model.evaluate(test_generator, verbose = 2)

In [ ]:
model.save('/home/davide/Documents/Progetto/dataset/ResNet50_FoodRecognition_RMSprop5_170621.h5')

In [ ]:
# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 143 layers and unfreeze the rest:
for layer in model.layers[:143]:
   layer.trainable = False
for layer in model.layers[143:]:
   layer.trainable = True

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(train_generator, epochs = 5, validation_data = valid_generator)

In [ ]:
model.save('/home/davide/Documents/Progetto/dataset/ResNet50_FoodRecognition_MRSprop5_Adam5_170621.h5')

In [ ]:
test_loss, test_acc = model.evaluate(test_generator, verbose = 2)